In [1]:
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.metrics import categorical_crossentropy
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from sklearn.metrics import confusion_matrix
    import itertools
    import os
    import shutil
    import random
    import glob
    import matplotlib.pyplot as plt
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    %matplotlib inline

In [2]:
df = pd.read_csv('planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')

In [3]:
df

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
...,...,...
40474,train_40474,clear primary
40475,train_40475,cloudy
40476,train_40476,agriculture clear primary
40477,train_40477,agriculture clear primary road


In [4]:
label = np.array(df['tags'])

In [5]:
os.getcwd()

'C:\\Users\\hp\\Desktop\\HAMOYE'

In [6]:
os.chdir('planet/train-jpg')

In [7]:
os.getcwd()

'C:\\Users\\hp\\Desktop\\HAMOYE\\planet\\train-jpg'

In [8]:
if os.path.isdir('train') is False:
    os.makedirs('train')
    os.makedirs('valid')
    os.makedirs('test')
    
    
    for c in random.sample(glob.glob('train*'), 25000):
        shutil.move(c, 'train')
    for c in random.sample(glob.glob('train*'), 10000):
        shutil.move(c, 'valid')
    for c in random.sample(glob.glob('train*'), 5487):
        shutil.move(c, 'test')


In [9]:
os.chdir('../../')

In [10]:
os.getcwd()

'C:\\Users\\hp\\Desktop\\HAMOYE'

In [11]:
train_path = 'planet/train-jpg/train'
valid_path = 'planet/train-jpg/valid'
test_path = 'planet/train-jpg/test'

In [27]:
df['tags'].unique()

array(['haze primary', 'agriculture clear primary water', 'clear primary',
       'agriculture clear habitation primary road', 'haze primary water',
       'agriculture clear cultivation primary water',
       'agriculture clear cultivation primary',
       'agriculture clear cultivation primary road',
       'agriculture clear primary slash_burn water',
       'clear primary water', 'cloudy', 'partly_cloudy primary',
       'agriculture clear primary road', 'clear primary road water',
       'conventional_mine partly_cloudy primary',
       'agriculture partly_cloudy primary water',
       'clear cultivation primary',
       'agriculture cultivation partly_cloudy primary',
       'agriculture clear primary', 'bare_ground haze',
       'bare_ground clear habitation primary',
       'agriculture clear habitation primary road water',
       'partly_cloudy primary road water',
       'artisinal_mine bare_ground clear primary water',
       'clear habitation primary road', 'blooming clear 

In [28]:
classes = ['haze primary', 'agriculture clear primary water', 'clear primary',
       'agriculture clear habitation primary road', 'haze primary water',
       'agriculture clear cultivation primary water',
       'agriculture clear cultivation primary',
       'agriculture clear cultivation primary road',
       'agriculture clear primary slash_burn water',
       'clear primary water', 'cloudy', 'partly_cloudy primary',
       'agriculture clear primary road', 'clear primary road water',
       'conventional_mine partly_cloudy primary',
       'agriculture partly_cloudy primary water',
       'clear cultivation primary',
       'agriculture cultivation partly_cloudy primary',
       'agriculture clear primary', 'bare_ground haze',
       'bare_ground clear habitation primary',
       'agriculture clear habitation primary road water',
       'partly_cloudy primary road water',
       'artisinal_mine bare_ground clear primary water',
       'clear habitation primary road', 'blooming clear primary',
       'agriculture partly_cloudy primary road',
       'agriculture clear primary road water',
       'blooming clear cultivation habitation primary slash_burn',
       'bare_ground clear water', 'partly_cloudy primary water',
       'agriculture bare_ground clear habitation primary road',
       'bare_ground clear primary',
       'agriculture partly_cloudy primary selective_logging',
       'agriculture partly_cloudy primary',
       'agriculture clear cultivation primary selective_logging',
       'clear primary road selective_logging water',
       'agriculture habitation partly_cloudy primary road',
       'agriculture artisinal_mine clear cultivation habitation primary road water',
       'artisinal_mine clear primary road', 'blow_down clear primary',
       'clear habitation road', 'bare_ground clear primary road',
       'agriculture clear cultivation habitation primary road',
       'agriculture clear cultivation habitation primary road water',
       'agriculture haze primary road water',
       'agriculture bare_ground habitation partly_cloudy primary road',
       'artisinal_mine clear primary road water',
       'bare_ground clear primary water', 'agriculture haze primary road',
       'clear primary blooming',
       'agriculture clear cultivation habitation primary road selective_logging',
       'clear habitation primary', 'clear cultivation habitation primary',
       'clear water', 'clear primary selective_logging',
       'agriculture artisinal_mine clear primary water',
       'clear primary road',
       'agriculture bare_ground clear cultivation habitation primary',
       'agriculture cultivation partly_cloudy primary road water',
       'bare_ground clear road', 'partly_cloudy road water',
       'agriculture clear cultivation habitation primary',
       'agriculture cultivation habitation partly_cloudy primary road',
       'agriculture haze primary',
       'partly_cloudy primary selective_logging',
       'agriculture clear habitation primary',
       'agriculture haze primary water',
       'clear habitation primary road water',
       'agriculture blooming clear cultivation primary',
       'agriculture partly_cloudy primary road water',
       'cultivation partly_cloudy primary water',
       'agriculture artisinal_mine clear primary road water',
       'partly_cloudy water',
       'agriculture habitation partly_cloudy primary',
       'agriculture bare_ground clear cultivation primary water',
       'agriculture clear cultivation habitation primary slash_burn',
       'clear cultivation habitation primary road',
       'bare_ground blow_down clear primary slash_burn',
       'agriculture cultivation partly_cloudy primary road',
       'agriculture cultivation partly_cloudy primary water',
       'agriculture clear cultivation primary road water',
       'agriculture blooming clear primary road water',
       'clear cultivation primary water',
       'blooming partly_cloudy primary', 'agriculture clear water',
       'agriculture clear cultivation primary road selective_logging',
       'agriculture clear habitation primary water',
       'cultivation partly_cloudy primary',
       'agriculture clear cultivation habitation primary road slash_burn',
       'clear primary road selective_logging',
       'habitation partly_cloudy road',
       'habitation partly_cloudy primary road',
       'bare_ground clear cultivation habitation primary water',
       'habitation haze primary road',
       'clear cultivation habitation primary water',
       'cultivation haze primary',
       'agriculture habitation partly_cloudy primary road water',
       'bare_ground clear primary road water',
       'clear habitation primary water',
       'artisinal_mine partly_cloudy primary water',
       'partly_cloudy primary road',
       'artisinal_mine clear primary road selective_logging water',
       'clear cultivation primary road water',
       'agriculture partly_cloudy',
       'agriculture clear primary road slash_burn',
       'artisinal_mine clear primary',
       'clear cultivation primary slash_burn',
       'partly_cloudy primary road selective_logging',
       'agriculture clear', 'bare_ground clear cultivation primary',
       'agriculture bare_ground clear cultivation primary',
       'agriculture clear primary slash_burn',
       'agriculture clear habitation primary road slash_burn',
       'clear primary selective_logging water',
       'agriculture artisinal_mine clear cultivation habitation primary water',
       'agriculture clear road', 'haze primary road',
       'haze primary road water', 'artisinal_mine clear primary water',
       'agriculture haze',
       'agriculture clear cultivation cultivation habitation primary road slash_burn water',
       'agriculture bare_ground clear habitation road',
       'agriculture habitation haze primary road',
       'cultivation habitation partly_cloudy primary',
       'clear cultivation habitation primary road water',
       'agriculture clear habitation road',
       'agriculture cultivation haze primary',
       'agriculture clear habitation primary slash_burn water',
       'agriculture bare_ground clear habitation primary',
       'blooming clear cultivation primary',
       'agriculture cultivation habitation partly_cloudy primary road water',
       'agriculture clear cultivation cultivation primary road',
       'partly_cloudy primary agriculture water',
       'agriculture cultivation haze primary road water',
       'partly_cloudy primary slash_burn',
       'cultivation partly_cloudy primary selective_logging',
       'bare_ground clear habitation primary road',
       'agriculture cultivation habitation haze primary road',
       'habitation haze primary',
       'bare_ground clear habitation primary water', 'bare_ground clear',
       'agriculture bare_ground clear habitation primary water',
       'agriculture cultivation habitation partly_cloudy primary',
       'agriculture clear cultivation habitation primary water',
       'agriculture cultivation cultivation haze primary road',
       'agriculture clear cultivation primary slash_burn',
       'artisinal_mine clear water', 'agriculture partly_cloudy road',
       'agriculture cultivation haze primary road',
       'haze primary road selective_logging', 'clear habitation water',
       'agriculture artisinal_mine bare_ground clear primary road water',
       'conventional_mine partly_cloudy primary road',
       'cultivation partly_cloudy primary slash_burn',
       'agriculture habitation haze primary',
       'cultivation habitation partly_cloudy primary road',
       'bare_ground habitation partly_cloudy primary road',
       'agriculture bare_ground cultivation haze primary water',
       'partly_cloudy primary blow_down',
       'agriculture cultivation haze primary water',
       'bare_ground clear conventional_mine primary road',
       'habitation partly_cloudy primary water',
       'bare_ground partly_cloudy primary',
       'agriculture cultivation partly_cloudy primary selective_logging',
       'habitation partly_cloudy primary road water',
       'cultivation habitation partly_cloudy primary road water',
       'agriculture clear cultivation cultivation primary',
       'artisinal_mine partly_cloudy primary road water',
       'bare_ground clear cultivation habitation primary road water',
       'clear cultivation primary road slash_burn',
       'agriculture habitation partly_cloudy primary water',
       'clear habitation road water',
       'agriculture clear cultivation water',
       'habitation haze primary water', 'agriculture haze road',
       'artisinal_mine bare_ground clear habitation primary road water',
       'agriculture haze water',
       'bare_ground clear cultivation primary road water',
       'bare_ground cultivation habitation partly_cloudy primary',
       'haze water', 'agriculture habitation haze primary water',
       'agriculture bare_ground clear primary',
       'habitation partly_cloudy primary',
       'agriculture clear primary road selective_logging',
       'agriculture bare_ground clear primary road',
       'agriculture partly_cloudy primary road slash_burn',
       'agriculture clear cultivation habitation road',
       'artisinal_mine clear habitation primary road water',
       'artisinal_mine clear habitation primary road',
       'clear cultivation primary road',
       'agriculture partly_cloudy primary road selective_logging',
       'clear bare_ground',
       'agriculture artisinal_mine clear primary road',
       'agriculture clear road water',
       'agriculture partly_cloudy primary blow_down',
       'agriculture clear primary road slash_burn water',
       'blooming clear primary water',
       'agriculture bare_ground partly_cloudy road',
       'artisinal_mine bare_ground clear primary selective_logging water',
       'bare_ground haze primary water',
       'cultivation habitation haze primary',
       'blow_down clear primary blow_down',
       'blooming clear primary selective_logging',
       'agriculture blow_down clear primary',
       'agriculture blooming clear habitation primary road',
       'clear conventional_mine habitation primary road',
       'clear primary slash_burn',
       'agriculture cultivation cultivation partly_cloudy primary road',
       'artisinal_mine haze primary',
       'agriculture bare_ground clear cultivation habitation primary road',
       'agriculture bare_ground clear conventional_mine habitation primary road',
       'clear conventional_mine primary water',
       'agriculture cultivation habitation partly_cloudy primary water',
       'conventional_mine haze primary',
       'agriculture blow_down clear habitation primary',
       'bare_ground clear habitation',
       'bare_ground cultivation haze primary',
       'bare_ground clear habitation road water',
       'blow_down partly_cloudy primary',
       'partly_cloudy primary selective_logging water',
       'agriculture clear cultivation primary selective_logging water',
       'agriculture cultivation habitation haze primary',
       'artisinal_mine bare_ground clear water',
       'habitation haze primary slash_burn',
       'agriculture bare_ground clear road',
       'habitation haze primary road water',
       'agriculture bare_ground haze', 'blooming clear primary road',
       'agriculture bare_ground haze primary road',
       'artisinal_mine bare_ground clear road water',
       'partly_cloudy primary blooming',
       'agriculture partly_cloudy water',
       'agriculture bare_ground clear habitation primary road water',
       'clear conventional_mine primary',
       'agriculture conventional_mine habitation partly_cloudy primary road',
       'clear cultivation primary road selective_logging',
       'agriculture blow_down clear cultivation primary',
       'agriculture bare_ground clear primary road water',
       'agriculture artisinal_mine clear road water',
       'agriculture artisinal_mine clear cultivation primary water',
       'clear cultivation primary selective_logging',
       'bare_ground haze primary',
       'agriculture clear cultivation cultivation habitation primary road',
       'agriculture clear primary selective_logging',
       'agriculture blow_down partly_cloudy primary blow_down',
       'agriculture bare_ground clear primary water',
       'agriculture clear habitation primary slash_burn',
       'bare_ground clear cultivation habitation primary road',
       'agriculture bare_ground blooming cultivation partly_cloudy primary selective_logging',
       'agriculture cultivation partly_cloudy primary road selective_logging',
       'agriculture cultivation cultivation partly_cloudy primary',
       'agriculture clear cultivation cultivation habitation road water',
       'agriculture clear cultivation cultivation habitation primary road water',
       'bare_ground clear primary slash_burn',
       'bare_ground clear cultivation primary water',
       'bare_ground habitation haze primary road',
       'bare_ground partly_cloudy',
       'bare_ground conventional_mine partly_cloudy primary road',
       'agriculture artisinal_mine clear conventional_mine cultivation habitation primary road water',
       'agriculture cultivation partly_cloudy primary slash_burn',
       'agriculture artisinal_mine clear habitation primary road water',
       'agriculture bare_ground clear',
       'agriculture bare_ground clear cultivation primary road',
       'agriculture clear cultivation habitation primary selective_logging',
       'bare_ground clear habitation road',
       'bare_ground clear habitation primary road water',
       'clear conventional_mine primary road',
       'agriculture clear primary conventional_mine',
       'clear primary blow_down', 'blow_down clear cultivation primary',
       'agriculture haze primary road selective_logging',
       'bare_ground cultivation partly_cloudy primary',
       'bare_ground clear primary selective_logging slash_burn water',
       'bare_ground habitation partly_cloudy primary',
       'artisinal_mine bare_ground clear primary road water',
       'agriculture clear conventional_mine primary road water',
       'bare_ground clear primary road selective_logging',
       'agriculture blooming clear cultivation primary selective_logging',
       'agriculture bare_ground partly_cloudy primary road',
       'cultivation haze primary water',
       'clear cultivation primary selective_logging water',
       'cultivation habitation haze primary water',
       'agriculture clear cultivation',
       'bare_ground blooming clear cultivation primary',
       'bare_ground clear primary selective_logging water',
       'haze primary blooming',
       'agriculture habitation partly_cloudy road',
       'agriculture clear cultivation primary blow_down',
       'agriculture clear cultivation road',
       'artisinal_mine bare_ground clear cultivation primary water',
       'bare_ground partly_cloudy primary road',
       'agriculture bare_ground clear cultivation primary slash_burn',
       'clear conventional_mine habitation primary',
       'bare_ground partly_cloudy primary water',
       'clear primary slash_burn water',
       'bare_ground partly_cloudy primary slash_burn',
       'agriculture clear cultivation cultivation primary road water',
       'agriculture habitation haze road',
       'artisinal_mine clear cultivation primary water',
       'clear conventional_mine primary road water',
       'agriculture partly_cloudy primary slash_burn',
       'agriculture bare_ground clear habitation',
       'agriculture cultivation cultivation habitation partly_cloudy primary road',
       'blow_down clear primary water',
       'clear habitation primary road selective_logging',
       'clear cultivation primary slash_burn water',
       'agriculture bare_ground partly_cloudy primary',
       'clear cultivation habitation primary slash_burn water',
       'agriculture bare_ground clear cultivation primary road water',
       'clear conventional_mine habitation primary road water',
       'agriculture cultivation habitation haze primary road water',
       'artisinal_mine partly_cloudy primary road',
       'conventional_mine habitation partly_cloudy primary road water',
       'agriculture clear habitation',
       'bare_ground clear cultivation primary road',
       'agriculture bare_ground clear conventional_mine habitation road',
       'clear road water',
       'agriculture artisinal_mine clear habitation road water',
       'agriculture clear cultivation habitation primary road slash_burn water',
       'agriculture clear cultivation primary road selective_logging slash_burn water',
       'conventional_mine habitation partly_cloudy primary road',
       'artisinal_mine clear road water',
       'agriculture clear cultivation cultivation road',
       'artisinal_mine bare_ground partly_cloudy primary road',
       'agriculture clear conventional_mine primary road',
       'clear conventional_mine habitation road',
       'partly_cloudy primary road water conventional_mine',
       'agriculture bare_ground haze primary',
       'bare_ground clear habitation primary selective_logging',
       'agriculture clear cultivation primary slash_burn water',
       'artisinal_mine clear habitation primary water',
       'artisinal_mine clear conventional_mine habitation primary road water',
       'habitation haze water', 'cultivation haze primary road',
       'haze primary selective_logging',
       'artisinal_mine clear cultivation primary road water',
       'partly_cloudy primary road selective_logging water',
       'agriculture cultivation habitation haze primary water',
       'bare_ground clear primary slash_burn water',
       'blow_down partly_cloudy primary water',
       'artisinal_mine haze primary water',
       'agriculture clear conventional_mine habitation primary road',
       'agriculture clear habitation primary road water conventional_mine',
       'agriculture bare_ground clear cultivation habitation primary road selective_logging',
       'clear habitation primary blooming',
       'clear cultivation habitation primary selective_logging',
       'cultivation partly_cloudy primary road',
       'agriculture blooming clear cultivation primary water',
       'agriculture blooming clear primary road',
       'artisinal_mine bare_ground partly_cloudy primary road water',
       'artisinal_mine partly_cloudy water',
       'haze primary road selective_logging water',
       'bare_ground blow_down clear primary',
       'agriculture artisinal_mine clear primary selective_logging water',
       'agriculture artisinal_mine bare_ground clear primary road',
       'blow_down clear primary slash_burn', 'haze road water',
       'agriculture habitation haze primary road water',
       'conventional_mine haze primary road',
       'bare_ground clear road water', 'partly_cloudy',
       'agriculture cultivation cultivation habitation partly_cloudy primary road water',
       'blooming haze primary', 'agriculture clear habitation road water',
       'agriculture clear cultivation cultivation habitation primary',
       'blooming cultivation partly_cloudy primary water',
       'agriculture clear cultivation primary blooming',
       'bare_ground cultivation partly_cloudy primary selective_logging',
       'agriculture bare_ground clear cultivation habitation primary road water',
       'agriculture conventional_mine partly_cloudy primary',
       'blow_down clear primary selective_logging',
       'blow_down clear cultivation habitation primary',
       'agriculture blooming clear primary',
       'agriculture bare_ground clear conventional_mine habitation primary road water',
       'agriculture artisinal_mine clear primary',
       'agriculture bare_ground habitation haze primary road',
       'artisinal_mine clear primary selective_logging', 'water',
       'clear habitation primary slash_burn',
       'blooming partly_cloudy primary selective_logging',
       'cultivation haze primary slash_burn',
       'bare_ground clear conventional_mine habitation primary road',
       'agriculture clear conventional_mine primary water',
       'agriculture partly_cloudy road water',
       'agriculture blooming blow_down clear cultivation primary',
       'agriculture blooming clear cultivation primary slash_burn',
       'agriculture cultivation habitation partly_cloudy primary slash_burn',
       'agriculture bare_ground clear water',
       'agriculture clear conventional_mine cultivation habitation primary road',
       'agriculture bare_ground clear habitation primary road slash_burn',
       'agriculture clear cultivation habitation primary conventional_mine',
       'agriculture cultivation partly_cloudy',
       'agriculture clear cultivation primary road slash_burn',
       'agriculture clear primary water conventional_mine',
       'artisinal_mine bare_ground clear primary',
       'artisinal_mine partly_cloudy primary',
       'clear habitation primary selective_logging',
       'agriculture bare_ground partly_cloudy primary water',
       'artisinal_mine clear conventional_mine primary road water',
       'agriculture clear primary road water conventional_mine',
       'agriculture clear primary blow_down',
       'agriculture clear cultivation cultivation primary slash_burn',
       'conventional_mine partly_cloudy',
       'agriculture artisinal_mine partly_cloudy primary water',
       'bare_ground clear conventional_mine habitation primary road water',
       'agriculture clear cultivation road water',
       'agriculture clear cultivation cultivation habitation primary road slash_burn',
       'bare_ground clear cultivation habitation primary',
       'agriculture bare_ground haze primary water',
       'agriculture clear conventional_mine cultivation habitation primary',
       'clear cultivation habitation primary slash_burn',
       'artisinal_mine clear primary selective_logging water',
       'agriculture clear habitation primary road selective_logging',
       'cultivation habitation haze primary road water',
       'agriculture clear primary road conventional_mine',
       'agriculture clear conventional_mine habitation road',
       'agriculture bare_ground clear cultivation road',
       'agriculture habitation partly_cloudy primary road water conventional_mine',
       'agriculture bare_ground habitation partly_cloudy primary',
       'artisinal_mine bare_ground partly_cloudy primary water',
       'clear primary road blooming',
       'agriculture clear habitation primary road selective_logging water',
       'clear cultivation habitation primary road slash_burn',
       'agriculture bare_ground clear cultivation habitation primary water',
       'agriculture blooming cultivation haze primary',
       'agriculture blooming clear cultivation primary road',
       'cultivation habitation partly_cloudy primary slash_burn',
       'agriculture artisinal_mine clear cultivation primary',
       'agriculture artisinal_mine clear cultivation primary road water',
       'agriculture clear cultivation cultivation habitation primary water',
       'clear road', 'bare_ground habitation haze primary water',
       'artisinal_mine bare_ground clear primary road',
       'agriculture blooming cultivation partly_cloudy primary selective_logging',
       'blow_down clear primary road',
       'bare_ground cultivation partly_cloudy primary road',
       'agriculture clear cultivation cultivation primary water',
       'agriculture clear habitation primary road blow_down',
       'agriculture blow_down partly_cloudy primary',
       'conventional_mine partly_cloudy primary road water',
       'clear primary road water conventional_mine',
       'artisinal_mine habitation partly_cloudy primary road water',
       'agriculture bare_ground clear cultivation primary selective_logging water',
       'agriculture blooming clear cultivation habitation primary',
       'artisinal_mine bare_ground clear habitation road',
       'agriculture cultivation cultivation partly_cloudy primary water',
       'bare_ground clear cultivation primary road selective_logging',
       'agriculture partly_cloudy primary road conventional_mine']

In [30]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=train_path, target_size=(224,224), classes= ['train'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes= ['train'],  batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=test_path, target_size=(224,224), classes= ['train'], batch_size=10,shuffle=False)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [22]:
arr = np.array(['dfd','frrrf','ffrrf'])

In [23]:
arr 

array(['dfd', 'frrrf', 'ffrrf'], dtype='<U5')